In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from pulp import *

In [2]:
candy = pd.read_csv('candy.csv')

Goal: create a basket of candies that minimizes cost and has

* an average deliciousness of 50%
* no more than 6% average fat per piece
* no more than 1.2% average salt per piece
* no more than 13% average sugar per piece
* contains at least some of each candy

In [3]:
candy

,Candy,Fat,Salt,Sugar,Deliciousness,Price
0,Reeses,0.20,0.02,0.15,0.82,0.64
1,Hersheys,0.09,0.01,0.09,0.64,0.55
2,Milky Way,0.07,0.02,0.11,0.71,0.61
3,3 Musketeers,0.05,0.02,0.18,0.50,0.45
4,Twix,0.20,0.03,0.16,0.90,0.69
5,Skittles,0.01,0.00,0.30,0.30,0.15
6,Starburst,0.03,0.00,0.20,0.34,0.21
7,Hubba Bubba,0.00,0.00,0.10,0.04,0.05


In [4]:
candy.mean()

Fat              0.08125
Salt             0.01250
Sugar            0.16125
Deliciousness    0.53125
Price            0.41875
dtype: float64

In [5]:
model = LpProblem("Halloween Basket Problem", LpMinimize)

In [6]:
reeses = LpVariable("Reeses", 1, None, LpInteger)
hersheys = LpVariable("Hersheys", 1, None, LpInteger)
milkyway = LpVariable("MilkyWay", 1, None, LpInteger)
musketeer = LpVariable("ThreeMusketeers", 1, None, LpInteger)
twix = LpVariable("Twix", 1, None, LpInteger)
skittles = LpVariable("Skittles", 1, None, LpInteger)
starburst = LpVariable("Starburst", 1, None, LpInteger)
hubbabubba = LpVariable("HubbaBubba", 1, None, LpInteger)

variables = [reeses, hersheys, milkyway, musketeer, twix, skittles, starburst, hubbabubba]

Objective Function

In [7]:
objective_function = []
for ix, price in enumerate(candy.Price):
    objective_function.append(price * variables[ix])
    
model += lpSum(objective_function), "Total Cost of Candies"

In [8]:
fat = []
for ix, f in enumerate(candy.Fat):
    fat.append(f * variables[ix])

salt = []
for ix, s in enumerate(candy.Salt):
    salt.append(s * variables[ix])    
    
sugar = []
for ix, su in enumerate(candy.Sugar):
    sugar.append(su * variables[ix])        
    
deliciousness = []
for ix, d in enumerate(candy.Deliciousness):
    deliciousness.append(d * variables[ix])

In [9]:
model += lpSum(deliciousness) >= 50, "Deliciousness Constraint"
model += lpSum(fat) <= 6, "Fat Constraint"
model += lpSum(salt) <= 1.2, "Salt Constraint"
model += lpSum(sugar) <= 13, "Sugar Constraint"

model += lpSum(variables) == 100, "Percentage Sum (full basket constraint)"

#for v in variables:
#    model += v > 0, f"{v.name} is in basket Constraint"

In [10]:
model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/brandonrose/opt/anaconda3/envs/b39/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/pp/vgfp1wf143q46m_8v6qbt3bw0000gn/T/5004c51d90d544fbb209ae26c7637fb4-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/pp/vgfp1wf143q46m_8v6qbt3bw0000gn/T/5004c51d90d544fbb209ae26c7637fb4-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10 COLUMNS
At line 71 RHS
At line 77 BOUNDS
At line 86 ENDATA
Problem MODEL has 5 rows, 8 columns and 36 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 40.6177 - 0.00 seconds
Cgl0004I processed model has 5 rows, 8 columns (8 integer (0 of which binary)) and 36 elements
Cutoff increment increased from 1e-05 to 0.00999
Cbc0012I Integer solution of 40.83 found by DiveCoefficient after 0 iterations and 0 nodes (0.02 se

1

In [12]:
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")
print('\n')

for var in model.variables():
    print(f"{var.name}: {var.value()}")

print('\n')
for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

status: 1, Optimal
objective: 40.75


Hersheys: 37.0
HubbaBubba: 12.0
MilkyWay: 20.0
Reeses: 1.0
Skittles: 1.0
Starburst: 27.0
ThreeMusketeers: 1.0
Twix: 1.0


Deliciousness_Constraint: 0.06000000000000183
Fat_Constraint: 3.3306690738754696e-16
Salt_Constraint: -0.3599999999999999
Sugar_Constraint: -0.07999999999999918
Percentage_Sum_(full_basket_constraint): 0.0
